<a href="https://colab.research.google.com/github/JullyVaz/etl-transacoes-bancarias-python/blob/main/SDW2025_ETL_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import time
import json
import pandas as pd
import requests
from google import genai
from google.colab import drive
from google.colab import userdata


# =========================
# CONFIG
# =========================
BASE_URL = "https://duke-rhodic-nondeductively.ngrok-free.dev"
headers = {"ngrok-skip-browser-warning": "true"}

GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

BATCH_SIZE = 10
saida = "/content/drive/MyDrive/mensagens_IA_gemini.csv"

# =========================
# DRIVE
# =========================
drive.mount("/content/drive")

# =========================
# FASE 1: EXTRACT
# =========================
print("========== FASE 1: EXTRACT ==========")

df = pd.read_csv("SDW2025_clean.csv")
print("✅ CSV lido com sucesso!")
print("Linhas:", df.shape[0], "| Colunas:", df.shape[1])

user_ids = df["UserID"].tolist()
print("📌 Total de IDs:", len(user_ids))

def get_user(user_id: int):
    r = requests.get(f"{BASE_URL}/users/{user_id}", headers=headers, timeout=10)
    return r.json() if r.status_code == 200 else None

users = []
for user_id in user_ids:
    user = get_user(int(user_id))
    if user:
        users.append(user)

print("✅ Usuários encontrados na API:", len(users))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
========== FASE 1: EXTRACT ==========
✅ CSV lido com sucesso!
Linhas: 100 | Colunas: 9
📌 Total de IDs: 100
✅ Usuários encontrados na API: 100


In [ ]:
print("========== FASE 2: TRANSFORM (GEMINI) ==========")
import os
import json
import time
import pandas as pd
import re
from google.colab import drive, userdata
from google import genai

# =========================
# DRIVE
# =========================
drive.mount("/content/drive")

saida = "/content/drive/MyDrive/mensagens_IA_gemini.csv"
BATCH_SIZE = 10

print("========== FASE 2: TRANSFORM (GEMINI) ==========")

# =========================
# CLIENT GEMINI (SECRET)
# =========================
api_key = userdata.get("GEMINI_API_KEY")
if not api_key:
    raise ValueError("❌ Secret GEMINI_API_KEY não encontrado. Vá em 'Secrets' e crie ele.")

client_gemini = genai.Client(api_key=api_key)

# =========================
# PREPARA DADOS
# =========================
users_ai = [{
    "id": int(u["id"]),
    "nome": u["nome"],
    "cidade": u["cidade"],
    "saldo": u["saldo"],
    "limite_cartao": u["limite_cartao"]
} for u in users]

# =========================
# RETOMADA / CONTINUAÇÃO
# =========================
if os.path.exists(saida):
    df_old = pd.read_csv(saida)
    processed_ids = set(df_old["UserID"].astype(int))
    resultado = df_old.to_dict(orient="records")
    print(f"✅ Continuando! Já existem {len(processed_ids)} salvos.")
else:
    processed_ids = set()
    resultado = []
    print("⚠️ Começando do zero...")

faltando = [u for u in users_ai if u["id"] not in processed_ids]
print("➡️ Faltando gerar:", len(faltando))

# =========================
# FUNÇÕES AUXILIARES
# =========================
def extrair_json(texto):
    """
    Remove blocos ```json e tenta extrair somente o array JSON [ ... ].
    """
    texto = (texto or "").strip()
    texto = texto.replace("```json", "").replace("```", "").strip()

    match = re.search(r"\[\s*{.*}\s*\]", texto, re.DOTALL)
    if match:
        return match.group(0)

    return texto


def gerar_lote_gemini(batch, tentativas=3):
    prompt = """
Você é um expert em marketing bancário do Santander.

Crie uma mensagem curta e amigável incentivando investimentos para cada cliente.

Regras:
- máximo 100 caracteres por mensagem
- português BR
- não prometer ganhos garantidos
- pode usar 1 emoji no máximo
- personalize com nome, cidade, saldo e limite do cartão

Retorne APENAS um JSON válido no formato:
[
  {"UserID": 1, "Mensagem": "texto..."},
  {"UserID": 2, "Mensagem": "texto..."}
]

Clientes:
"""
    for u in batch:
        prompt += (
            f'- UserID:{u["id"]} | Nome:{u["nome"]} | Cidade:{u["cidade"]} '
            f'| Saldo:{u["saldo"]} | Limite:{u["limite_cartao"]}\n'
        )

    for tentativa in range(1, tentativas + 1):
        try:
            resp = client_gemini.models.generate_content(
                model="models/gemini-2.0-flash",  # ✅ MODELO CORRETO
                contents=prompt
            )

            content = extrair_json(resp.text)
            data = json.loads(content)

            # normaliza e limita
            saida_final = []
            for item in data:
                uid = int(item["UserID"])
                msg = str(item["Mensagem"]).replace("\n", " ").strip()[:100]
                saida_final.append({"UserID": uid, "Mensagem": msg})

            return saida_final

        except Exception as e:
            print(f"⚠️ Erro no Gemini (tentativa {tentativa}/{tentativas}): {e}")
            time.sleep(2)

    raise RuntimeError("❌ Falhou após várias tentativas ao gerar lote com Gemini.")

# =========================
# LOOP POR LOTES
# =========================
for i in range(0, len(faltando), BATCH_SIZE):
    batch = faltando[i:i+BATCH_SIZE]
    ids = [x["id"] for x in batch]
    print(f"\n🚀 Gerando lote: {ids}")

    res = gerar_lote_gemini(batch)

    for item in res:
        uid = int(item["UserID"])
        msg = item["Mensagem"][:100]

        nome = next((x["nome"] for x in batch if x["id"] == uid), "Cliente")

        resultado.append({"UserID": uid, "Nome": nome, "Mensagem": msg})
        print(f"✅ User {uid}: {msg}")

    df_out = pd.DataFrame(resultado).drop_duplicates(subset=["UserID"]).sort_values("UserID")
    df_out.to_csv(saida, index=False)
    print(f"💾 Salvo: {saida} | Total: {len(df_out)}")

    time.sleep(2)

print("\n🏁 FINALIZADO! CSV Gemini pronto:", saida)


========== FASE 2: TRANSFORM (GEMINI) ==========
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
========== FASE 2: TRANSFORM (GEMINI) ==========
⚠️ Começando do zero...
➡️ Faltando gerar: 100

🚀 Gerando lote: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
✅ User 1: Gabriela, que tal investir parte dos seus R$45317? 😉 Seu futuro agradece! Santander no Rio.
✅ User 2: Renata, com seu limite de R$26422 em SP, que tal fazer seu saldo render mais? Santander.
✅ User 3: Fernanda, invista parte dos R$24631 e veja seu dinheiro crescer! 😉 Santander em Porto Alegre.
✅ User 4: Thiago, que tal começar a investir? Conte com o Santander em Curitiba! Seu limite é R$4886.
✅ User 5: Beatriz, Santander em Curitiba: faça seus R$23433 renderem ainda mais! Consulte opções.
✅ User 6: Rafaela, aproveite seu limite de R$22898 e invista no futuro! Santander em BH te ajuda.
✅ User 7: Rafael, que tal fazer seus R$5940 renderem mais no Rio? 😉 Consul

In [5]:
import pandas as pd
import requests
import time

print("========== FASE 3: LOAD (API) ==========")

BASE_URL = "https://duke-rhodic-nondeductively.ngrok-free.dev"
headers = {"ngrok-skip-browser-warning": "true"}

saida = "/content/drive/MyDrive/mensagens_IA_gemini.csv"
df = pd.read_csv(saida)

print("✅ CSV carregado:", saida)
print("Linhas:", len(df))

success = 0
fail = 0

for _, row in df.iterrows():
    user_id = int(row["UserID"])
    msg = str(row["Mensagem"]).strip()

    payload = {"description": msg}

    try:
        r = requests.post(
            f"{BASE_URL}/users/{user_id}/news",
            json=payload,
            headers=headers,
            timeout=10
        )

        if r.status_code in [200, 201]:
            success += 1
            print(f"✅ User {user_id} OK")
        else:
            fail += 1
            print(f"❌ User {user_id} ERRO {r.status_code}: {r.text[:120]}")

    except Exception as e:
        fail += 1
        print(f"❌ User {user_id} EXCEÇÃO: {e}")

    time.sleep(0.3)

print("\n🏁 FINALIZADO!")
print("Sucesso:", success)
print("Falhas:", fail)



========== FASE 3: LOAD (API) ==========
✅ CSV carregado: /content/drive/MyDrive/mensagens_IA_gemini.csv
Linhas: 100
✅ User 1 OK
✅ User 2 OK
✅ User 3 OK
✅ User 4 OK
✅ User 5 OK
✅ User 6 OK
✅ User 7 OK
✅ User 8 OK
✅ User 9 OK
✅ User 10 OK
✅ User 11 OK
✅ User 12 OK
✅ User 13 OK
✅ User 14 OK
✅ User 15 OK
✅ User 16 OK
✅ User 17 OK
✅ User 18 OK
✅ User 19 OK
✅ User 20 OK
✅ User 21 OK
✅ User 22 OK
✅ User 23 OK
✅ User 24 OK
✅ User 25 OK
✅ User 26 OK
✅ User 27 OK
✅ User 28 OK
✅ User 29 OK
✅ User 30 OK
✅ User 31 OK
✅ User 32 OK
✅ User 33 OK
✅ User 34 OK
✅ User 35 OK
✅ User 36 OK
✅ User 37 OK
✅ User 38 OK
✅ User 39 OK
✅ User 40 OK
✅ User 41 OK
✅ User 42 OK
✅ User 43 OK
✅ User 44 OK
✅ User 45 OK
✅ User 46 OK
✅ User 47 OK
✅ User 48 OK
✅ User 49 OK
✅ User 50 OK
✅ User 51 OK
✅ User 52 OK
✅ User 53 OK
✅ User 54 OK
✅ User 55 OK
✅ User 56 OK
✅ User 57 OK
✅ User 58 OK
✅ User 59 OK
✅ User 60 OK
✅ User 61 OK
✅ User 62 OK
✅ User 63 OK
✅ User 64 OK
✅ User 65 OK
✅ User 66 OK
✅ User 67 OK
✅ User 68 OK
✅ User 6

In [6]:
import pandas as pd
import requests
import time

print("========== FASE 3.1: CHECK (VALIDAÇÃO) ==========")
# (Opcional) Validação: confirma se todos os usuários receberam news


# =========================
# CONFIG
# =========================
BASE_URL = "https://duke-rhodic-nondeductively.ngrok-free.dev"
headers = {"ngrok-skip-browser-warning": "true"}

csv_mensagens = "/content/drive/MyDrive/mensagens_IA_gemini.csv"
saida_check = "/content/drive/MyDrive/check_envio_news.csv"

# =========================
# LER CSV COM IDs
# =========================
df = pd.read_csv(csv_mensagens)
df["UserID"] = df["UserID"].astype(int)

print("✅ CSV carregado:", len(df), "usuários")

resultados = []

ok = 0
sem_news = 0
erro = 0

# =========================
# CHECAR CADA USER NA API
# =========================
for uid in df["UserID"].tolist():
    try:
        r = requests.get(f"{BASE_URL}/users/{uid}", headers=headers, timeout=10)

        if r.status_code != 200:
            erro += 1
            resultados.append({
                "UserID": uid,
                "Status": "ERRO_API",
                "HTTP": r.status_code,
                "QtdNews": None
            })
            print(f"⚠️ User {uid} erro HTTP {r.status_code}")
            continue

        data = r.json()
        news = data.get("news", [])
        qtd = len(news)

        if qtd > 0:
            ok += 1
            status = "OK"
        else:
            sem_news += 1
            status = "SEM_NEWS"

        resultados.append({
            "UserID": uid,
            "Status": status,
            "HTTP": 200,
            "QtdNews": qtd
        })

        print(f"User {uid} -> {status} | news={qtd}")

    except Exception as e:
        erro += 1
        resultados.append({
            "UserID": uid,
            "Status": "EXCECAO",
            "HTTP": None,
            "QtdNews": None,
            "Erro": str(e)[:120]
        })
        print(f"❌ User {uid} exceção: {str(e)[:120]}")

    time.sleep(0.2)

# =========================
# SALVAR RELATÓRIO
# =========================
df_check = pd.DataFrame(resultados).sort_values("UserID")
df_check.to_csv(saida_check, index=False)

print("\n========== RESUMO ==========")
print("✅ OK (tem news):", ok)
print("❌ SEM_NEWS:", sem_news)
print("⚠️ ERROS:", erro)
print("\n📄 Relatório salvo em:", saida_check)


========== FASE 3.1: CHECK (VALIDAÇÃO) ==========
✅ CSV carregado: 100 usuários
User 1 -> OK | news=9
User 2 -> OK | news=5
User 3 -> OK | news=6
User 4 -> OK | news=5
User 5 -> OK | news=5
User 6 -> OK | news=4
User 7 -> OK | news=4
User 8 -> OK | news=4
User 9 -> OK | news=4
User 10 -> OK | news=4
User 11 -> OK | news=4
User 12 -> OK | news=4
User 13 -> OK | news=4
User 14 -> OK | news=4
User 15 -> OK | news=4
User 16 -> OK | news=4
User 17 -> OK | news=4
User 18 -> OK | news=4
User 19 -> OK | news=4
User 20 -> OK | news=4
User 21 -> OK | news=4
User 22 -> OK | news=4
User 23 -> OK | news=4
User 24 -> OK | news=4
User 25 -> OK | news=4
User 26 -> OK | news=4
User 27 -> OK | news=4
User 28 -> OK | news=4
User 29 -> OK | news=4
User 30 -> OK | news=4
User 31 -> OK | news=4
User 32 -> OK | news=4
User 33 -> OK | news=4
User 34 -> OK | news=4
User 35 -> OK | news=4
User 36 -> OK | news=4
User 37 -> OK | news=4
User 38 -> OK | news=4
User 39 -> OK | news=4
User 40 -> OK | news=4
User 41 -